In [ ]:
# import sys

# !{sys.executable} -m pip install -U "transformers>=4.40.0" "datasets" "evaluate" pandas scikit-learn joblib tqdm



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# import sys

# # 1. Снести старый torch
# !{sys.executable} -m pip uninstall -y torch torchvision torchaudio

# # 2. Поставить свежий torch с CUDA (12.6)
# !{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126



Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.9.0%2Bcu126-cp311-cp311-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.24.0%2Bcu126-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchaudio-2.9.0%2Bcu126-cp311-cp311-win_amd64.whl.metadata (7.0 kB)
Using cached https://download.pytorch.org/whl/cu126/torch-2.9.0%2Bcu126-cp311-cp311-win_amd64.whl (2582.7 MB)
Using cached https://download.pytorch.org/whl/cu126/torchv


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch

print("Версия torch:", torch.__version__)
print("CUDA доступна?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Версия torch: 2.6.0+cu118
CUDA доступна? True
GPU: NVIDIA GeForce RTX 4060 Ti


In [ ]:
Версия torch: 2.9.0+cu126
CUDA доступна? True
GPU: NVIDIA GeForce RTX 3060 Laptop GPU

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # <-- добавили для FocalLoss
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import joblib

# ===== НАСТРОЙКИ =====
# Сейчас ты дообучаешь уже сохранённую модель
MODEL_NAME = "ai-forever/ruBert-base"

CSV_PATH = r"C:\Users\botoh\Downloads\Telegram Desktop\raw_precleaned (2).csv"  # путь к твоему файлу
TEXT_COL = "text"    # колонка с текстом
LABEL_COL = "label"  # колонка с метками

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используем устройство:", device)

# ===== 1. ЧИТАЕМ ДАННЫЕ =====
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=[TEXT_COL, LABEL_COL])

print("Всего строк в датасете:", len(df))

# кодируем метки в числа
le = LabelEncoder()
df["label_id"] = le.fit_transform(df[LABEL_COL])

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=52,
    stratify=df["label_id"]
)

num_labels = df["label_id"].nunique()
print("Число классов:", num_labels)

# ===== 2. DATASET =====
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class LettersDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=256):
        self.texts = df[TEXT_COL].astype(str).tolist()
        self.labels = df["label_id"].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        enc = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(label, dtype=torch.long)
        return item

train_dataset = LettersDataset(train_df, tokenizer, max_length=256)
val_dataset = LettersDataset(val_df, tokenizer, max_length=256)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print("Train size:", len(train_dataset), "Val size:", len(val_dataset))

# ===== 3. МОДЕЛЬ, ОПТИМАЙЗЕР, SCHEDULER =====
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels
).to(device)

# ---- FOCAL LOSS ----
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        """
        gamma > 0 усиливает фокус на сложных примерах.
        alpha можно задать как тензор весов классов (по умолчанию None).
        """
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        inputs: логиты (batch_size, num_classes)
        targets: индексы классов (batch_size,)
        """
        logpt = F.log_softmax(inputs, dim=-1)   # лог-вероятности
        pt = torch.exp(logpt)                   # вероятности

        # (1 - p)^gamma * log(p)
        focal_logpt = (1 - pt) ** self.gamma * logpt

        loss = F.nll_loss(
            focal_logpt,
            targets,
            weight=self.alpha,
            reduction=self.reduction
        )
        return loss

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = FocalLoss(gamma=2.0)  # <-- используем FocalLoss вместо CrossEntropy

num_epochs = 3
total_steps = len(train_loader) * num_epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# ===== 4. ФУНКЦИИ ОБУЧЕНИЯ И ВАЛИДАЦИИ =====
def train_one_epoch(model, loader, optimizer, scheduler, criterion, device):
    model.train()
    total_loss = 0.0

    for batch in loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        # ВАЖНО: не передаём labels в модель, чтобы она не считала свой CE-loss
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        logits = outputs.logits

        # Считаем Focal Loss поверх логитов
        loss = criterion(logits, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    return total_loss / len(loader)


def evaluate(model, loader, device, le):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # f1-macro и твой score
    f1_macro = f1_score(all_labels, all_preds, average="macro")

    labels_unique = np.unique(all_labels)
    f1_per_class = f1_score(all_labels, all_preds, average=None, labels=labels_unique)
    variance = np.var(f1_per_class)
    score = f1_macro - 0.1 * np.sqrt(variance)

    # отчёт с текстовыми метками
    true_txt = le.inverse_transform(all_labels)
    pred_txt = le.inverse_transform(all_preds)

    print("\nКлассификационный отчёт (валидация):")
    print(classification_report(true_txt, pred_txt, digits=4))
    print("F1-macro:", f1_macro)
    print("F1 по классам:", dict(zip(le.inverse_transform(labels_unique), f1_per_class)))
    print("Дисперсия F1 по классам:", variance)
    print("Твой score = f1_macro - 0.1 * sqrt(variance) =", score)

    return f1_macro, variance, score

# ===== 5. ЦИКЛ ОБУЧЕНИЯ =====
best_score = -1e9

for epoch in range(1, num_epochs + 1):
    print(f"\n===== Эпоха {epoch}/{num_epochs} =====")
    train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, criterion, device)
    print("Train loss:", train_loss)

    # сначала считаем метрики на валидации
    f1_macro, variance, score = evaluate(model, val_loader, device, le)

    # потом логируем их в файл
    with open("logs.txt", "a", encoding="utf-8") as f:
        f.write(f"\nЭпоха {epoch}\n")
        f.write(f"{f1_macro:.6f} - f1_macro\n{variance:.6f} - дисперсия\n{score:.6f} - score\n")
        f.write("-" * 50 + "\n")

    if score > best_score:
        best_score = score
        print("Новый лучший score! Сохраняем модель...")
        model.save_pretrained("./rubert_letters_best")
        tokenizer.save_pretrained("./rubert_letters_best")
        joblib.dump(le, "./rubert_letters_best/label_encoder.joblib")

print("\nОбучение завершено. Лучший score:", best_score)


Используем устройство: cuda
Всего строк в датасете: 1380
Число классов: 36
Train size: 1104 Val size: 276


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===== Эпоха 1/3 =====
Train loss: 2.835017497124879

Классификационный отчёт (валидация):
                                                                              precision    recall  f1-score   support

                                                       Блок бизнес-директора     0.0000    0.0000    0.0000         1
                                                      Блок деректора по газу     0.0000    0.0000    0.0000        11
                                          Блок директора по газовым проектам     0.0000    0.0000    0.0000         5
                                                 Блок директора по мощностям     0.0000    0.0000    0.0000        38
                                                 Блок директора по персоналу     0.0000    0.0000    0.0000         3
                                                  Блок директора по портфелю     0.0000    0.0000    0.0000         4
                                            Блок директора по проектированию     0

C:\Users\botoh\PycharmProjects\BERTA\.venv1\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\botoh\PycharmProjects\BERTA\.venv1\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\botoh\PycharmProjects\BERTA\.venv1\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capital


===== Эпоха 2/3 =====
Train loss: 2.470520105914793

Классификационный отчёт (валидация):
                                                                              precision    recall  f1-score   support

                                                       Блок бизнес-директора     0.0000    0.0000    0.0000         1
                                                      Блок деректора по газу     0.0000    0.0000    0.0000        11
                                          Блок директора по газовым проектам     0.0000    0.0000    0.0000         5
                                                 Блок директора по мощностям     0.3750    0.8684    0.5238        38
                                                 Блок директора по персоналу     0.0000    0.0000    0.0000         3
                                                  Блок директора по портфелю     0.0000    0.0000    0.0000         4
                                            Блок директора по проектированию     0

C:\Users\botoh\PycharmProjects\BERTA\.venv1\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\botoh\PycharmProjects\BERTA\.venv1\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\botoh\PycharmProjects\BERTA\.venv1\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capital


===== Эпоха 3/3 =====
Train loss: 2.0451303392216778

Классификационный отчёт (валидация):
                                                                              precision    recall  f1-score   support

                                                       Блок бизнес-директора     0.0000    0.0000    0.0000         1
                                                      Блок деректора по газу     0.0000    0.0000    0.0000        11
                                          Блок директора по газовым проектам     0.0000    0.0000    0.0000         5
                                                 Блок директора по мощностям     0.4762    0.7895    0.5941        38
                                                 Блок директора по персоналу     0.0000    0.0000    0.0000         3
                                                  Блок директора по портфелю     0.0000    0.0000    0.0000         4
                                            Блок директора по проектированию     

C:\Users\botoh\PycharmProjects\BERTA\.venv1\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\botoh\PycharmProjects\BERTA\.venv1\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\botoh\PycharmProjects\BERTA\.venv1\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capital


Обучение завершено. Лучший score: 0.08333627665721116
